In [54]:
# @title Upload CSV
import pandas as pd
import random

hari_df = pd.read_csv('data_skripsi_hari.csv')
ruang_df = pd.read_csv('data_skripsi_ruang.csv')
jam_df = pd.read_csv('data_skripsi_jam.csv')
dosen_df = pd.read_csv('data_skripsi_dosen.csv')
mk_genap_df = pd.read_csv('data_skripsi_mk_genap.csv')
data_dosen_df = pd.read_csv('data_skripsi_data_dosen.csv')

In [55]:
# @title Gabungkan data_dosen dengan dosen
merged_df = pd.merge(
    pd.merge(data_dosen_df, dosen_df, on='id_dosen'),
    mk_genap_df, on='id_mk_genap'
)

print("Data Gabungan:")
print(merged_df.head())

total_sks = merged_df['sks'].sum()
print(f"Total slot SKS yang dibutuhkan: {total_sks}")
print(f"Total data dalam merged_df: {len(merged_df)}")

Data Gabungan:
   id_dosen  id_mk_genap kelas                   nama_dosen  \
0         2    211840231     A  Ahmad Azhari, S.Kom., M.Eng   
1         2    211861331     A  Ahmad Azhari, S.Kom., M.Eng   
2         2    211861431     A  Ahmad Azhari, S.Kom., M.Eng   
3         3    211840131     A     Ali Tarmuji, S.T., M.Cs.   
4         3    211860330     A     Ali Tarmuji, S.T., M.Cs.   

                              nama_mk_genap  smt  sks    sifat kategori  \
0                          Grafika Komputer    4    3    Wajib        -   
1                Pengembangan Aplikasi Game    6    3  Pilihan       SC   
2                          Pengenalan Pola     6    3  Pilihan       SC   
3  Analisis dan Perancangan Perangkat Lunak    4    3    Wajib        -   
4                  Rekayasa Perangkat Lunak    6    3    Wajib        -   

    metode  
0  Offline  
1  Offline  
2   Online  
3  Offline  
4  Offline  
Total slot SKS yang dibutuhkan: 282
Total data dalam merged_df: 107


Tahapan GWO

1. Preprocessing
    
    Membangun struktur slot waktu.

2. Inisialisasi Populasi (GWO)
    

    Inisialisasi populasi serigala acak (Xi).

    Setiap "serigala" dalam GWO mewakili solusi penjadwalan yang mungkin (misalnya: variasi pengaturan slot).

3. Fitness Function (GWO)
    Hitung nilai fitness untuk setiap serigala.

    Mengevaluasi kualitas penjadwalan (misalnya: minimalisasi konflik, kepadatan ruang, dll).

4. Proses Optimasi (GWO)
    Tentukan Alpha, Beta, dan Delta berdasarkan nilai fitness.

    Menggunakan hierarki Alpha, Beta, Delta untuk memperbarui posisi solusi.

    While (iterasi < maksimum iterasi):
      1. Perbarui parameter a, A, dan C.
      2. For setiap serigala:
          1. Perbarui posisi serigala berdasarkan Alpha, Beta, dan Delta.
      3. Hitung nilai fitness untuk setiap serigala.
      4. Perbarui Alpha, Beta, dan Delta.
5. Postprocessing
    Return solusi terbaik (Alpha).
    
    Menampilkan jadwal terbaik hasil optimasi.


In [56]:
# @title Preprosessing
hari_list = hari_df['nama_hari'].tolist()
ruang_list = ruang_df['nama_ruang'].tolist()
jam_list = jam_df[['id_jam', 'jam_awal', 'jam_akhir']].to_dict('records')
mata_kuliah_list = mk_genap_df.set_index('id_mk_genap').to_dict('index')

def slot_generator():
    wolf = []
    for id_counter, (hari, ruang, jam) in enumerate(
        [(h, r, j) for h in hari_list for r in ruang_list for j in jam_list], start=1
    ):
        wolf.append({
            "id_slot": id_counter,
            "mata_kuliah": None,
            "dosen": None,
            "ruang": ruang,
            "hari": hari,
            "jam_mulai": jam['jam_awal'],
            "jam_selesai": jam['jam_akhir'],
            "kelas": None,
            "sks": None,
            "metode": None
        })
    return wolf

slots = slot_generator()
for slot in slots[:5]:
    print(slot)

print("\n")
for slot in slots[-5:]:
    print(slot)

{'id_slot': 1, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:00:00', 'jam_selesai': '7:50:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 2, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:50:00', 'jam_selesai': '8:45:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 3, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '8:45:00', 'jam_selesai': '9:35:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 4, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '9:35:00', 'jam_selesai': '10:25:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 5, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '10:30:00', 'jam_selesai': '11:20:00', 'kelas': None, 'sks': None, 'metode': None}


{'id_slot': 500, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.63', 'hari'

In [60]:
import random
from collections import defaultdict
from datetime import datetime

def time_to_minutes(t):
    try:
        dt = datetime.strptime(t, "%H:%M:%S")
    except ValueError:
        dt = datetime.strptime(t, "%H:%M")
    return dt.hour * 60 + dt.minute

def create_random_schedule():
    # Ambil slot dari fungsi slot_generator()
    schedule = slot_generator()
    # Acak urutan data matakuliah dari hasil merge
    merged_shuffled = merged_df.sample(frac=1).iterrows()

    courses_inserted = 0  # Untuk menghitung jumlah mata kuliah yang berhasil ditempatkan
    
    # Struktur untuk tracking konflik
    room_allocations = defaultdict(list)    # key: (ruang, hari) -> list of (start, end)
    teacher_allocations = defaultdict(list)  # key: (dosen, hari) -> list of (start, end)
    class_allocations = defaultdict(list)    # key: (kelas, hari) -> list of (start, end)
    
    for idx, row in merged_shuffled:
        mata_kuliah = row['nama_mk_genap']
        dosen = row['nama_dosen']
        kelas = row['kelas']
        sks = int(row['sks'])
        metode = row['metode']
        
        # Tentukan semua kemungkinan posisi untuk blok slot sesuai jumlah SKS
        possible_positions = list(range(len(schedule) - sks + 1))
        random.shuffle(possible_positions)
        placed = False
        
        # Coba penempatan normal: cari blok slot yang memenuhi kriteria tanpa konflik
        for i in possible_positions:
            block = schedule[i:i+sks]
            
            # Pastikan seluruh slot dalam blok masih kosong
            if not all(slot['mata_kuliah'] is None for slot in block):
                continue
            
            # Validasi: seluruh slot dalam blok harus berada pada hari yang sama
            if not all(slot['hari'] == block[0]['hari'] for slot in block):
                continue
            
            # Untuk kelas offline, seluruh slot harus berada pada ruang yang sama
            if metode != 'Online' and not all(slot['ruang'] == block[0]['ruang'] for slot in block):
                continue
            
            hari = block[0]['hari']
            ruang = 'Online' if metode == 'Online' else block[0]['ruang']
            start_time = time_to_minutes(block[0]['jam_mulai'])
            end_time = time_to_minutes(block[-1]['jam_selesai'])
            
            # 1. Cek konflik ruang (hanya untuk kelas offline)
            if metode != 'Online':
                if any(existing_start < end_time and existing_end > start_time 
                       for (existing_start, existing_end) in room_allocations[(ruang, hari)]):
                    continue
            
            # 2. Cek konflik dosen (baik online maupun offline)
            if any(existing_start < end_time and existing_end > start_time 
                   for (existing_start, existing_end) in teacher_allocations[(dosen, hari)]):
                continue
            
            # 3. Cek konflik kelas
            if any(existing_start < end_time and existing_end > start_time 
                   for (existing_start, existing_end) in class_allocations[(kelas, hari)]):
                continue
            
            for slot in block:
                slot.update({
                    "mata_kuliah": mata_kuliah,
                    "dosen": dosen,
                    "kelas": kelas,
                    "sks": sks,
                    "metode": metode,
                    "ruang": ruang
                })
            
            time_block = (start_time, end_time)
            if metode != 'Online':
                room_allocations[(ruang, hari)].append(time_block)
            teacher_allocations[(dosen, hari)].append(time_block)
            class_allocations[(kelas, hari)].append(time_block)
            
            placed = True
            break
        
        if not placed:
            min_conflict = float('inf')
            best_index = None
            best_block = None
            
            for i in possible_positions:
                block = schedule[i:i+sks]
                if not all(slot['hari'] == block[0]['hari'] for slot in block):
                    continue
                if metode != 'Online' and not all(slot['ruang'] == block[0]['ruang'] for slot in block):
                    continue
                
                hari = block[0]['hari']
                ruang = 'Online' if metode == 'Online' else block[0]['ruang']
                start_time = time_to_minutes(block[0]['jam_mulai'])
                end_time = time_to_minutes(block[-1]['jam_selesai'])
                
                conflict = 0
                if metode != 'Online':
                    for (existing_start, existing_end) in room_allocations[(ruang, hari)]:
                        if existing_start < end_time and existing_end > start_time:
                            conflict += 1
                for (existing_start, existing_end) in teacher_allocations[(dosen, hari)]:
                    if existing_start < end_time and existing_end > start_time:
                        conflict += 1
                for (existing_start, existing_end) in class_allocations[(kelas, hari)]:
                    if existing_start < end_time and existing_end > start_time:
                        conflict += 1
                
                if conflict < min_conflict:
                    min_conflict = conflict
                    best_index = i
                    best_block = block
            
            if best_block is not None:
                hari = best_block[0]['hari']
                ruang = 'Online' if metode == 'Online' else best_block[0]['ruang']
                start_time = time_to_minutes(best_block[0]['jam_mulai'])
                end_time = time_to_minutes(best_block[-1]['jam_selesai'])
                
                for slot in best_block:
                    slot.update({
                        "mata_kuliah": mata_kuliah,
                        "dosen": dosen,
                        "kelas": kelas,
                        "sks": sks,
                        "metode": metode,
                        "ruang": ruang
                    })
                
                time_block = (start_time, end_time)
                if metode != 'Online':
                    room_allocations[(ruang, hari)].append(time_block)
                teacher_allocations[(dosen, hari)].append(time_block)
                class_allocations[(kelas, hari)].append(time_block)
                
                placed = True
            else:
                print(f"Gagal menempatkan (tidak ada blok sama sekali): {mata_kuliah} - {dosen} - {kelas}")
        
        if placed:
            courses_inserted += 1
    
    # Tampilkan validasi jumlah data yang telah dimasukkan
    total_courses = len(merged_df)
    print(f"\nTotal mata kuliah yang berhasil dimasukkan: {courses_inserted} / {total_courses}")
    
    return schedule

# Contoh penggunaan
population_size = 1
population = [create_random_schedule() for _ in range(population_size)]

for i in range(population_size):
    for slot in population[i]:
        print(slot)



Total mata kuliah yang berhasil dimasukkan: 107 / 107
{'id_slot': 1, 'mata_kuliah': 'Rekayasa Perangkat Lunak', 'dosen': 'Ali Tarmuji, S.T., M.Cs.', 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:00:00', 'jam_selesai': '7:50:00', 'kelas': 'A', 'sks': 3, 'metode': 'Offline'}
{'id_slot': 2, 'mata_kuliah': 'Rekayasa Perangkat Lunak', 'dosen': 'Ali Tarmuji, S.T., M.Cs.', 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:50:00', 'jam_selesai': '8:45:00', 'kelas': 'A', 'sks': 3, 'metode': 'Offline'}
{'id_slot': 3, 'mata_kuliah': 'Rekayasa Perangkat Lunak', 'dosen': 'Ali Tarmuji, S.T., M.Cs.', 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '8:45:00', 'jam_selesai': '9:35:00', 'kelas': 'A', 'sks': 3, 'metode': 'Offline'}
{'id_slot': 4, 'mata_kuliah': 'Grafika Komputer', 'dosen': 'Ahmad Azhari, S.Kom., M.Eng', 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '9:35:00', 'jam_selesai': '10:25:00', 'kelas': 'A', 'sks': 3, 'metode': 'Offline'}
{'id_slot': 5, 'mata_kuliah': 'Grafik

Konstrain berat 1
Konstrain ringan 0.5

Konstrain berat:
dosen tidak boleh mengajar mata kuliah/kelas berbeda pada jam yang sama
ruang kelas tidak boleh digunakan lebih dari 1 kelas/dosen
3 sks membutuhkan 3 slot waktu
ruangan antar kelas harus sama
slot waktu harus berurutan

Konstrain ringan:
kelas dijadwalkan pada waktu paling akhir dan paling awal pada hari itu

In [58]:
from collections import defaultdict
from datetime import datetime

def time_to_minutes(t):
    try:
        dt = datetime.strptime(t, "%H:%M:%S")
    except ValueError:
        dt = datetime.strptime(t, "%H:%M")
    return dt.hour * 60 + dt.minute

def minutes_to_time(mins):
    hours = mins // 60
    minutes = mins % 60
    return f"{hours:02d}:{minutes:02d}"

def calculate_fitness(schedule):
    max_score = 100
    penalty = 0

    teacher_conflicts = []  # Hard constraint: dosen mengajar dua kelas dalam waktu bersamaan
    room_conflicts = []     # Hard constraint: ruang dipakai lebih dari satu kelas secara bersamaan

    # Struktur tracking
    teacher_intervals = defaultdict(list)  # key: (dosen, hari) -> list of (start, end)
    room_intervals = defaultdict(list)       # key: (ruang, hari) -> list of (start, end)
    teacher_daily_courses = defaultdict(set) # key: (dosen, hari) -> set of mata kuliah
    class_edge_times = defaultdict(lambda: {'min': float('inf'), 'max': -float('inf')})  # key: (kelas, hari)

    # Kumpulkan informasi dari setiap slot yang terisi
    for slot in schedule:
        if not slot['mata_kuliah']:
            continue

        day = slot['hari']
        teacher = slot['dosen']
        room = slot['ruang']
        kelas = slot['kelas']
        mk = slot['mata_kuliah']
        start = time_to_minutes(slot['jam_mulai'])
        end = time_to_minutes(slot['jam_selesai'])
        
        # Hard constraint: track interval dosen dan ruang
        teacher_intervals[(teacher, day)].append((start, end))
        if room != "Online":
            room_intervals[(room, day)].append((start, end))
        
        # Soft constraint: dosen mengajar lebih dari satu mata kuliah di hari yang sama
        teacher_daily_courses[(teacher, day)].add(mk)
        
        # Soft constraint: catat waktu awal dan akhir tiap kelas
        if start < class_edge_times[(kelas, day)]['min']:
            class_edge_times[(kelas, day)]['min'] = start
        if start > class_edge_times[(kelas, day)]['max']:
            class_edge_times[(kelas, day)]['max'] = start

    # Cek konflik dosen (hard constraint)
    for (teacher, day), intervals in teacher_intervals.items():
        intervals.sort(key=lambda x: x[0])
        for i in range(1, len(intervals)):
            prev = intervals[i-1]
            curr = intervals[i]
            if curr[0] < prev[1]:
                teacher_conflicts.append({
                    'teacher': teacher,
                    'day': day,
                    'conflict': (prev, curr)
                })
                penalty += 1

    # Cek konflik ruang (hard constraint)
    for (room, day), intervals in room_intervals.items():
        intervals.sort(key=lambda x: x[0])
        for i in range(1, len(intervals)):
            prev = intervals[i-1]
            curr = intervals[i]
            if curr[0] < prev[1]:
                room_conflicts.append({
                    'room': room,
                    'day': day,
                    'conflict': (prev, curr)
                })
                penalty += 1

    # Soft constraint: dosen mengajar lebih dari satu mata kuliah dalam satu hari
    for (teacher, day), courses in teacher_daily_courses.items():
        if len(courses) > 1:
            penalty += 0.5 * (len(courses) - 1)

    # Soft constraint: penalti untuk kelas yang dijadwalkan pada slot paling awal atau paling akhir hari
    # (misal, penalti 0.5 untuk masing-masing ujung hari)
    for (kelas, day), bounds in class_edge_times.items():
        if bounds['min'] != float('inf'):
            penalty += 0.5  # penalti untuk kelas yang memulai pada jam paling awal
        if bounds['max'] != -float('inf'):
            penalty += 0.5  # penalti untuk kelas yang berada di slot akhir

    score = max(0, max_score - penalty)
    
    return {
        'score': score,
        'penalty': penalty,
        'violations': {
            'hard_teacher': len(teacher_conflicts),
            'hard_room': len(room_conflicts),
            'soft_teacher_multiple': sum(0.5*(len(courses)-1) for courses in teacher_daily_courses.values() if len(courses) > 1),
            'soft_class_edge': len(class_edge_times)
        },
        'teacher_conflicts': teacher_conflicts,
        'room_conflicts': room_conflicts
    }

# Contoh penggunaan
population_size = 1
population = [create_random_schedule() for _ in range(population_size)]

for idx, schedule in enumerate(population):
    fitness = calculate_fitness(schedule)
    print(f"\nIndividu {idx+1}:")
    # for slot in schedule:
    #     print(slot)
    print(f"\nFitness Score: {fitness['score']}")
    print(f"Total Penalty: {fitness['penalty']}")
    
    print("\nDetail Konflik Dosen:")
    if not fitness['teacher_conflicts']:
        print(" - Tidak ada konflik dosen.")
    else:
        for conflict in fitness['teacher_conflicts']:
            t1 = minutes_to_time(conflict['conflict'][0][0])
            t2 = minutes_to_time(conflict['conflict'][0][1])
            t3 = minutes_to_time(conflict['conflict'][1][0])
            t4 = minutes_to_time(conflict['conflict'][1][1])
            print(f" - Dosen {conflict['teacher']} pada {conflict['day']}: {t1}-{t2} bertabrakan dengan {t3}-{t4}")
    
    print("\nDetail Konflik Ruang:")
    if not fitness['room_conflicts']:
        print(" - Tidak ada konflik ruang.")
    else:
        for conflict in fitness['room_conflicts']:
            t1 = minutes_to_time(conflict['conflict'][0][0])
            t2 = minutes_to_time(conflict['conflict'][0][1])
            t3 = minutes_to_time(conflict['conflict'][1][0])
            t4 = minutes_to_time(conflict['conflict'][1][1])
            print(f" - Ruang {conflict['room']} pada {conflict['day']}: {t1}-{t2} bertabrakan dengan {t3}-{t4}")
    
    print("\nPelanggaran Soft Constraint Lain:")
    print(f" - Dosen mengajar multi mata kuliah/hari: {fitness['violations']['soft_teacher_multiple']}")
    print(f" - Kelas di slot ujung hari: {fitness['violations']['soft_class_edge']}")



Total mata kuliah yang berhasil dimasukkan: 107 / 107

Individu 1:

Fitness Score: 59.0
Total Penalty: 41.0

Detail Konflik Dosen:
 - Dosen Guntur  Maulana Zamroni, B.Sc., M.Kom. pada Rabu: 16:10-17:00 bertabrakan dengan 16:10-17:00
 - Dosen Guntur  Maulana Zamroni, B.Sc., M.Kom. pada Rabu: 17:00-17:50 bertabrakan dengan 17:00-17:50

Detail Konflik Ruang:
 - Tidak ada konflik ruang.

Pelanggaran Soft Constraint Lain:
 - Dosen mengajar multi mata kuliah/hari: 5.0
 - Kelas di slot ujung hari: 34


In [59]:

# Operator modifikasi sederhana: swap dua penugasan (random swap)
def random_swap(schedule):
    new_schedule = [slot.copy() for slot in schedule]
    assigned_indices = [i for i, slot in enumerate(new_schedule) if slot['mata_kuliah'] is not None]
    if len(assigned_indices) < 2:
        return new_schedule
    i, j = random.sample(assigned_indices, 2)
    # Swap seluruh informasi penugasan antara dua slot
    new_schedule[i]['mata_kuliah'], new_schedule[j]['mata_kuliah'] = new_schedule[j]['mata_kuliah'], new_schedule[i]['mata_kuliah']
    new_schedule[i]['dosen'], new_schedule[j]['dosen'] = new_schedule[j]['dosen'], new_schedule[i]['dosen']
    new_schedule[i]['kelas'], new_schedule[j]['kelas'] = new_schedule[j]['kelas'], new_schedule[i]['kelas']
    new_schedule[i]['sks'], new_schedule[j]['sks'] = new_schedule[j]['sks'], new_schedule[i]['sks']
    new_schedule[i]['metode'], new_schedule[j]['metode'] = new_schedule[j]['metode'], new_schedule[i]['metode']
    new_schedule[i]['ruang'], new_schedule[j]['ruang'] = new_schedule[j]['ruang'], new_schedule[i]['ruang']
    return new_schedule

# Final function GWO
def gwo(num_iterations, population_size):
    # Inisialisasi populasi
    population = [create_random_schedule() for _ in range(population_size)]
    fitness_list = [calculate_fitness(schedule)['score'] for schedule in population]
    
    # Identifikasi Alpha, Beta, Delta (solusi terbaik, kedua, dan ketiga)
    sorted_population = sorted(zip(population, fitness_list), key=lambda x: x[1], reverse=True)
    alpha = sorted_population[0][0]
    beta = sorted_population[1][0] if population_size > 1 else alpha
    delta = sorted_population[2][0] if population_size > 2 else beta
    
    for iter in range(num_iterations):
        a = 2 - iter * (2 / num_iterations)  # Parameter a menurun secara linear
        new_population = []
        for candidate in population:
            # Lakukan modifikasi kandidat dengan operator random_swap
            candidate_new = random_swap(candidate)
            # Opsional: dengan probabilitas tertentu (dipengaruhi a), lakukan swap tambahan
            if random.random() < a / 2:
                candidate_new = random_swap(candidate_new)
            # Jika fitness solusi baru lebih baik, terima solusi baru
            if calculate_fitness(candidate_new)['score'] > calculate_fitness(candidate)['score']:
                new_population.append(candidate_new)
            else:
                new_population.append(candidate)
        population = new_population
        fitness_list = [calculate_fitness(schedule)['score'] for schedule in population]
        sorted_population = sorted(zip(population, fitness_list), key=lambda x: x[1], reverse=True)
        alpha = sorted_population[0][0]
        beta = sorted_population[1][0] if population_size > 1 else alpha
        delta = sorted_population[2][0] if population_size > 2 else beta
        print(f"Iteration {iter+1}: Best Fitness = {sorted_population[0][1]}")
    
    best_schedule = alpha
    return alpha, beta, delta, best_schedule

# Contoh penggunaan GWO
num_iterations = 20
population_size = 50
alpha, beta, delta, best_schedule = gwo(num_iterations, population_size)

print("\nHasil Akhir GWO:")
# print("Alpha (Solusi terbaik):")
# for slot in alpha:
#     print(slot)
# print("\nBeta (Solusi kedua terbaik):")
# for slot in beta:
#     print(slot)
# print("\nDelta (Solusi ketiga terbaik):")
# for slot in delta:
#     print(slot)
print("\nJadwal Terbaik (Alpha):")
for slot in best_schedule:
    print(slot)


Total mata kuliah yang berhasil dimasukkan: 107 / 107

Total mata kuliah yang berhasil dimasukkan: 107 / 107

Total mata kuliah yang berhasil dimasukkan: 107 / 107

Total mata kuliah yang berhasil dimasukkan: 107 / 107

Total mata kuliah yang berhasil dimasukkan: 107 / 107

Total mata kuliah yang berhasil dimasukkan: 107 / 107

Total mata kuliah yang berhasil dimasukkan: 107 / 107

Total mata kuliah yang berhasil dimasukkan: 107 / 107

Total mata kuliah yang berhasil dimasukkan: 107 / 107

Total mata kuliah yang berhasil dimasukkan: 107 / 107

Total mata kuliah yang berhasil dimasukkan: 107 / 107

Total mata kuliah yang berhasil dimasukkan: 107 / 107

Total mata kuliah yang berhasil dimasukkan: 107 / 107

Total mata kuliah yang berhasil dimasukkan: 107 / 107

Total mata kuliah yang berhasil dimasukkan: 107 / 107

Total mata kuliah yang berhasil dimasukkan: 107 / 107

Total mata kuliah yang berhasil dimasukkan: 107 / 107

Total mata kuliah yang berhasil dimasukkan: 107 / 107

Total mat